# Spreadsheet
## Dask

In [1]:
# import numpy as np
# import pandas as pd

In [2]:
import dask.dataframe as dd

# import dask.array as da
# import dask.bag as db

### 서울시 세종문화회관 공연 및 전시 정보

In [3]:
df = dd.read_csv(
    "../data/sejongpac_performs.csv",
    dtype={
        "공연시작일": "object",
        "공연종료일": "object",
        "썸네일이미지": "object",
        "출연진 소개": "object",
        "프로그램 정보": "object",
    },
)

In [4]:
df.head()

,공연코드,공연명,공연시작일,공연종료일,장소,시간,연령,기획사,문의전화,티켓정보,할인정보,유료할인율,장르,게시여부,게시여부명,썸네일이미지,공연개요,프로그램 정보,출연진 소개
0,b1dt2206091514a01,(TEST) 전시연동테스트 공연_세종미술관,20250601,20250630,세종미술관 1관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기획전시,Y,공개,NaN,NaN,NaN,NaN
1,dt2206091515a01,(TEST) 전시연동테스트 공연_상상톡톡미술관,20250601,20250630,꿈의숲 상상톡톡미술관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,기획전시,Y,공개,NaN,NaN,NaN,NaN
2,chpi2204141747a01,2022 서울시향 실내악 시리즈 VII,20221126,20221126,세종체임버홀,오후 5시,초등학생 이상 관람 가능,서울시향,서울시향 1588-1210,R석 50000 / S석 30000 / A석 10000,<html><head></head><body spellcheck='false'><p...,NaN,기악,Y,공개,NaN,출연바이올린 정지혜 한지연 김민정 Jihye Chung Jiyun Han Minju...,NaN,NaN
3,chpi2204141743a01,2022 서울시향 실내악 시리즈VI,20220917,20220917,세종체임버홀,오후 5시,초등학생 이상 관람 가능,서울시향,서울시향 1588-1210,R석 50000 / S석 30000 / A석 10000,<html><head></head><body spellcheck='false'><p...,NaN,기악,Y,공개,NaN,출연바이올린 웨인 린 신아라 최해성 김민정Wayne Lin Arah Shin Hae...,NaN,NaN
4,chpi2205022005a01,2022 Seoul Philharmonic Orchestra Chamber Musi...,20220917,20220917,세종체임버홀,17:00,Preschool children are not allowed,Seoul Philharmonic Orchestra,1588-1210,R 50000won / S 30000won / A 10000won,NaN,NaN,기악,Y,공개,NaN,Program IntroductionThis chamber music program...,NaN,NaN
